In [2]:
import pandas as pd
import numpy as np
import os
import sys
import gzip
import shutil
import json
import re
import tmalign_mapping
import Bio.PDB as bpdb

In [19]:
filepath_sample = "/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/3hqa.cif"
parser = bpdb.MMCIFParser()

In [20]:
structure=parser.get_structure("3hqa", filepath_sample)

In [21]:
res_list= bpdb.Selection.unfold_entities(structure, "C")
print(res_list)

[<Chain id=A>, <Chain id=B>]


In [22]:
structure[0]['B'].get_residues()
res_list= bpdb.Selection.unfold_entities(structure[0]['B'], "R")
print(res_list)
# print(structure[0]['B'][2])
print(structure[0]['A'][1])

[<Residue GLN het=  resseq=3 icode= >, <Residue LYS het=  resseq=4 icode= >, <Residue LYS het=  resseq=5 icode= >, <Residue ILE het=  resseq=6 icode= >, <Residue GLU het=  resseq=7 icode= >, <Residue GLU het=  resseq=8 icode= >, <Residue ILE het=  resseq=9 icode= >, <Residue ALA het=  resseq=10 icode= >, <Residue ALA het=  resseq=11 icode= >, <Residue LYS het=  resseq=12 icode= >, <Residue TYR het=  resseq=13 icode= >, <Residue LYS het=  resseq=14 icode= >, <Residue HIS het=  resseq=15 icode= >, <Residue SER het=  resseq=16 icode= >, <Residue VAL het=  resseq=17 icode= >, <Residue VAL het=  resseq=18 icode= >, <Residue LYS het=  resseq=19 icode= >, <Residue LYS het=  resseq=20 icode= >, <Residue CYS het=  resseq=21 icode= >, <Residue CYS het=  resseq=22 icode= >, <Residue TYR het=  resseq=23 icode= >, <Residue ASP het=  resseq=24 icode= >, <Residue GLY het=  resseq=25 icode= >, <Residue ALA het=  resseq=26 icode= >, <Residue CYS het=  resseq=27 icode= >, <Residue VAL het=  resseq=28 ic

In [23]:
ppb= bpdb.PPBuilder()
polypeptides = ppb.build_peptides(structure)

In [24]:
for pp in polypeptides:
    print(pp.get_sequence())

MLQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANIS
QKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANIS


In [29]:
structure_2 = parser.get_structure("3hqb", "/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/3hqb.cif")
polypeptides_2 = ppb.build_peptides(structure_2)
for x in ppb.build_peptides(structure_2):
    print(x.get_sequence())


MLQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
LQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA


In [58]:
print(str(polypeptides[1].get_sequence()))
print(str(polypeptides_2[1].get_sequence()))

QKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANIS
LQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA


In [36]:
print(structure[0]['A'][64])
print(structure_2[0]['A'][1])

<Residue ASN het=  resseq=64 icode= >
<Residue MET het=  resseq=1 icode= >


In [63]:
class GlySelect(bpdb.Select):
    def accept_residue(self, residue):
        if residue.id[1] < 64 and residue.parent.id == 'A':
            return 1
        elif residue.id[1] <=62 and residue.parent.id == 'B':
            return 1
        else:
            return 0

In [64]:
io=bpdb.PDBIO()
io.set_structure(structure)
io.save("trial.pdb", GlySelect())

In [65]:
parser_2 = bpdb.PDBParser()
new_str = parser_2.get_structure("lol", "trial.pdb")
for x in ppb.build_peptides(new_str):
    print(x.get_sequence())

MLQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
QKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLR


In [56]:
res_list= bpdb.Selection.unfold_entities(new_str, "R")
print(res_list)

[<Residue GLY het=  resseq=25 icode= >, <Residue GLY het=  resseq=44 icode= >, <Residue GLY het=  resseq=25 icode= >, <Residue GLY het=  resseq=44 icode= >]


In [66]:
class GllolSelect(bpdb.Select):
    def accept_residue(self, residue):
        if residue.parent.id == 'A':
            return 1
        elif residue.id[1] > 1 and residue.parent.id == 'B':
            return 1
        else:
            return 0

In [67]:
io=bpdb.PDBIO()
io.set_structure(structure_2)
io.save("trial_2.pdb", GllolSelect())